# Customer Segmentation - RFM Analysis + K-Means

## Objective
- Analyze customer behavior using RFM (Recency, Frequency, Monetary)
- Apply K-Means clustering for customer segmentation
- Run locally with pandas + sklearn, connecting to Databricks via SQL Connector

## Dataset
- Source: Olist Brazilian E-commerce (Gold layer)
- Tables: `dim_order`, `fct_order_payment`

---
## 1. Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import configparser

from databricks import sql

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
import seaborn as sns

---
## 2. Configuration & Connection

In [ ]:
databricks_cfg = configparser.ConfigParser()
databricks_cfg.read(Path.home() / ".databrickscfg")

DATABRICKS_HOST = databricks_cfg["DEFAULT"]["host"].rstrip("/")
DATABRICKS_TOKEN = databricks_cfg["DEFAULT"]["token"]
DATABRICKS_HTTP_PATH = "/sql/1.0/warehouses/956f28a1a104215e"

CATALOG = "olist_project"
GOLD_SCHEMA = "gold"

print(f"DATABRICKS_HOST: {DATABRICKS_HOST}")
print(f"CATALOG: {CATALOG}")
print(f"GOLD_SCHEMA: {GOLD_SCHEMA}")

In [ ]:
def get_connection():
    return sql.connect(
        server_hostname=DATABRICKS_HOST.replace("https://", ""),
        http_path=DATABRICKS_HTTP_PATH,
        access_token=DATABRICKS_TOKEN
    )

with get_connection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT 1")
    result = cursor.fetchone()
    print(f"Connection test: {result[0]} (expected: 1)")
    cursor.close()

print("Databricks connection successful!")

---
## 3. Load Data from Gold Tables

In [ ]:
print("Loading dim_order...")
with get_connection() as conn:
    orders_pd = pd.read_sql(
        f"""
        SELECT DISTINCT 
            order_id, 
            customer_unique_id, 
            order_purchase_timestamp
        FROM {CATALOG}.{GOLD_SCHEMA}.dim_order 
        WHERE is_current = TRUE AND is_delivered = TRUE
        """,
        conn
    )
print(f"Orders: {len(orders_pd):,} rows")

In [ ]:
print("Loading fct_order_payment...")
with get_connection() as conn:
    order_payments_pd = pd.read_sql(
        f"SELECT order_id, payment_value FROM {CATALOG}.{GOLD_SCHEMA}.fct_order_payment",
        conn
    )
print(f"Order Payments: {len(order_payments_pd):,} rows")

In [ ]:
print("=== Orders ===")
display(orders_pd.head(3))

print("\n=== Order Payments ===")
display(order_payments_pd.head(3))

---
## 4. Feature Engineering - RFM Calculation

**RFM Analysis** is a proven customer segmentation technique that evaluates customers based on 3 behavioral metrics:

| Metric | Definition | Interpretation |
|--------|------------|----------------|
| **Recency (R)** | Days since last purchase | Lower = Better (recent buyer) |
| **Frequency (F)** | Number of unique orders | Higher = Better (repeat buyer) |
| **Monetary (M)** | Total spending amount | Higher = Better (high spender) |

Customers with **low Recency**, **high Frequency**, and **high Monetary** are the most valuable.

In [ ]:
orders_pd['order_purchase_timestamp'] = pd.to_datetime(orders_pd['order_purchase_timestamp'])
max_date = orders_pd['order_purchase_timestamp'].max()
print(f"Reference date (max order date): {max_date}")

In [ ]:
orders_with_payments = orders_pd.merge(
    order_payments_pd[['order_id', 'payment_value']],
    on='order_id',
    how='inner'
)
print(f"Orders with payments: {len(orders_with_payments):,} rows")

In [ ]:
rfm_df = orders_with_payments.groupby('customer_unique_id').agg(
    recency_days=('order_purchase_timestamp', lambda x: (max_date - x.max()).days),
    frequency=('order_id', 'nunique'),
    monetary=('payment_value', 'sum')
).reset_index()

print(f"RFM DataFrame: {len(rfm_df):,} unique customers")
rfm_df.head(10)

In [ ]:
rfm_df.describe()

---
## 5. Exploratory Data Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(rfm_df['recency_days'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Recency Distribution')
axes[0].set_xlabel('Days since last purchase')
axes[0].set_ylabel('Count')

axes[1].hist(rfm_df['frequency'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Frequency Distribution')
axes[1].set_xlabel('Number of orders')
axes[1].set_ylabel('Count')

axes[2].hist(rfm_df['monetary'], bins=50, edgecolor='black', alpha=0.7)
axes[2].set_title('Monetary Distribution')
axes[2].set_xlabel('Total spending (BRL)')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].boxplot(rfm_df['recency_days'])
axes[0].set_title('Recency Boxplot')

axes[1].boxplot(rfm_df['frequency'])
axes[1].set_title('Frequency Boxplot')

axes[2].boxplot(rfm_df['monetary'])
axes[2].set_title('Monetary Boxplot')

plt.tight_layout()
plt.show()

In [ ]:
correlation_matrix = rfm_df[['recency_days', 'frequency', 'monetary']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('RFM Correlation Matrix')
plt.show()

---
## 6. Preprocessing

- Replace 0 with 1 for all RFM values (avoid log(0))
- Log transformation for Frequency and Monetary
- StandardScaler to normalize features

In [ ]:
rfm_df['recency_days'] = rfm_df['recency_days'].replace(0, 1)
rfm_df['frequency'] = rfm_df['frequency'].replace(0, 1)
rfm_df['monetary'] = rfm_df['monetary'].replace(0, 1)

rfm_df['frequency_log'] = np.log10(rfm_df['frequency'])
rfm_df['monetary_log'] = np.log10(rfm_df['monetary'])

rfm_df.head()

In [ ]:
feature_cols = ['recency_days', 'frequency_log', 'monetary_log']
X = rfm_df[feature_cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Feature matrix shape: {X_scaled.shape}")
print(f"Mean after scaling: {X_scaled.mean(axis=0)}")
print(f"Std after scaling: {X_scaled.std(axis=0)}")

---
## 7. Elbow Method - Optimal Cluster Selection

In [ ]:
k_values = range(2, 11)
inertia_values = []
silhouette_values = []

for k in k_values:
    print(f"Training K-Means with k={k}...")
    kmeans = KMeans(n_clusters=k, random_state=5, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(X_scaled)
    inertia_values.append(kmeans.inertia_)
    sil_score = silhouette_score(X_scaled, labels)
    silhouette_values.append(sil_score)
    print(f"  k={k}: Inertia={kmeans.inertia_:.2f}, Silhouette={sil_score:.4f}")

print("\nElbow analysis completed!")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(list(k_values), inertia_values, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (k)')
axes[0].set_ylabel('Inertia (WCSS)')
axes[0].set_title('Elbow Method')
axes[0].grid(True, alpha=0.3)

axes[1].plot(list(k_values), silhouette_values, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (k)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Score vs k')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

best_k_silhouette = list(k_values)[np.argmax(silhouette_values)]
print(f"\nBest k based on Silhouette Score: {best_k_silhouette}")
print(f"Max Silhouette Score: {max(silhouette_values):.4f}")

In [ ]:
OPTIMAL_K = 4
print(f"Selected optimal k: {OPTIMAL_K}")

---
## 8. Train Final Model

In [ ]:
kmeans_final = KMeans(n_clusters=OPTIMAL_K, random_state=5, n_init=10, max_iter=300)
rfm_df['cluster_id'] = kmeans_final.fit_predict(X_scaled)

inertia_final = kmeans_final.inertia_
silhouette_final = silhouette_score(X_scaled, rfm_df['cluster_id'])
n_customers = len(rfm_df)

print(f"Model trained successfully!")
print(f"Inertia: {inertia_final:.2f}")
print(f"Silhouette Score: {silhouette_final:.4f}")
print(f"Total customers: {n_customers:,}")

---
## 9. Cluster Analysis & Segment Naming

In [ ]:
cluster_profile = rfm_df.groupby('cluster_id').agg(
    count=('customer_unique_id', 'count'),
    avg_recency=('recency_days', 'mean'),
    avg_frequency=('frequency', 'mean'),
    avg_monetary=('monetary', 'mean')
).reset_index()

cluster_profile = cluster_profile.sort_values('cluster_id')
cluster_profile['percentage'] = (cluster_profile['count'] / cluster_profile['count'].sum() * 100).round(1)
cluster_profile

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].bar(cluster_profile['cluster_id'], cluster_profile['avg_recency'], color='skyblue', edgecolor='black')
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('Avg Recency (days)')
axes[0].set_title('Average Recency by Cluster')

axes[1].bar(cluster_profile['cluster_id'], cluster_profile['avg_frequency'], color='lightgreen', edgecolor='black')
axes[1].set_xlabel('Cluster')
axes[1].set_ylabel('Avg Frequency')
axes[1].set_title('Average Frequency by Cluster')

axes[2].bar(cluster_profile['cluster_id'], cluster_profile['avg_monetary'], color='salmon', edgecolor='black')
axes[2].set_xlabel('Cluster')
axes[2].set_ylabel('Avg Monetary (BRL)')
axes[2].set_title('Average Monetary by Cluster')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.pie(
    cluster_profile['count'], 
    labels=[f"Cluster {i}" for i in cluster_profile['cluster_id']],
    autopct='%1.1f%%',
    startangle=90
)
plt.title('Customer Distribution by Cluster')
plt.show()

### Segment Naming & Business Insights

Based on RFM cluster profiles, we identify 4 distinct customer segments:

| Segment | % | Profile | Business Insight | Recommended Action |
|---------|---|---------|------------------|--------------------|
| **Churned Customers** | 29% | 400+ days inactive, low spend | Former customers who stopped buying | Send discounts to bring them back |
| **New Customers** | 38% | Recent purchase, single order, low spend | First-time buyers exploring the platform | Encourage 2nd purchase |
| **Loyal Customers** | 3% | Multiple orders (~2x), high total spend | Most valuable repeat customers | Give rewards to retain them |
| **High Spenders** | 30% | Recent, single order, high spend (~319 BRL) | Big spenders with growth potential | Suggest products to increase repeat orders |

In [ ]:
SEGMENT_MAPPING = {
    0: "Churned Customers",
    1: "New Customers",
    2: "Loyal Customers",
    3: "High Spenders"
}

rfm_df['segment_name'] = rfm_df['cluster_id'].map(SEGMENT_MAPPING)

print("Segment Distribution:")
for cluster_id, segment_name in SEGMENT_MAPPING.items():
    count = len(rfm_df[rfm_df['cluster_id'] == cluster_id])
    pct = count / len(rfm_df) * 100
    print(f"  {segment_name}: {count:,} customers ({pct:.1f}%)")

In [ ]:
segment_counts = rfm_df['segment_name'].value_counts()

plt.figure(figsize=(10, 6))
colors = ['#66b3ff', '#99ff99', '#ffcc99', '#ff9999']
plt.barh(segment_counts.index, segment_counts.values, color=colors, edgecolor='black')
plt.xlabel('Number of Customers')
plt.title('Customer Segment Distribution')

for i, v in enumerate(segment_counts.values):
    plt.text(v + 500, i, f'{v:,}', va='center')

plt.tight_layout()
plt.show()

---
## 10. Summary

### Customer Segments

| Segment | % | Strategy |
|---------|---|----------|
| **New Customers** | 38% | Encourage 2nd purchase |
| **High Spenders** | 30% | Suggest products to increase repeat orders |
| **Churned Customers** | 29% | Send discounts to bring them back |
| **Loyal Customers** | 3% | Give rewards to retain them |

### Key Takeaways

1. **67% of customers are one-time buyers** (New + Churned) → Focus on retention strategies
2. **Only 3% are repeat buyers** (Loyal Customers) → High opportunity to grow this segment
3. **29% have churned** → Significant revenue recovery opportunity with win-back campaigns
4. **30% are high spenders** → Convert them to loyal customers for long-term value

In [ ]:
print("="*60)
print("CUSTOMER SEGMENTATION - SUMMARY")
print("="*60)
print(f"\nModel: K-Means (k={OPTIMAL_K})")
print(f"Silhouette Score: {silhouette_final:.4f}")
print(f"Total customers: {n_customers:,}")
print("\nSegment Distribution:")
print("-"*60)

for _, row in cluster_profile.iterrows():
    segment = SEGMENT_MAPPING[row['cluster_id']]
    print(f"{segment}:")
    print(f"  Count: {row['count']:,} ({row['percentage']}%)")
    print(f"  Avg R: {row['avg_recency']:.0f} days | Avg F: {row['avg_frequency']:.2f} | Avg M: {row['avg_monetary']:.2f} BRL")

print("="*60)